In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from pprint import pprint
import pandas as pd
import skimage.io
import sys

import time
from tqdm import tqdm
import cProfile

sys.path.insert(0,'..')

# custom packages
from preprocessing.utils.data_loader import PandasDataLoader
from preprocessing.generators import TiffGenerator
from preprocessing.utils.tiling import tile, get_tile_coords
from utils.utils import set_gpu_memory, seed_all

In [3]:
set_gpu_memory()
!nvidia-smi

2 Physical GPUs, 1 Logical GPUs
Tue Jun  2 23:51:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla M40           On   | 00000000:04:00.0 Off |                    0 |
| N/A   33C    P0    59W / 250W |    111MiB / 11448MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M40           On   | 00000000:82:00.0 Off |                    0 |
| N/A   24C    P8    15W / 250W |     11MiB / 11448MiB |      0%      Default |
+-------------------------------+----------------------+

In [4]:
# ------------------
# Directories
# ------------------
DATA_DIR = '../data'  # General path to the data dir
IMG_DIR = '../data/train_images'  # Path to the TILED images
TRAIN_MASKS_DIR = '../data/masks'  # Path to the masks

# ------------------
# Data constants
# ------------------
NFOLDS = 4  # number of folds to use for training/validation (cross validation)
SEED=5  # the seed TODO: REPLACE THIS WITH A FUNCTION THAT SEEDS EVERYTHING WITH SEED
TRAIN_FOLD=0  # select the first fold for training/validation

# ------------------
# Network parameters
# ------------------
# some of these parameters might disappear in a future release
SZ = 256  # width and heigth of the image
NUM_CLASSES = 6 
BATCH_SIZE = 8
NUM_EPOCHS = 15  
NUM_TILES = 16 # this number cannot be changed freely in the png tile generator
LEARNING_RATE = 1e-3
TILE_LEVEL = 1

tile_params = {'N': NUM_TILES, 'sz': SZ}


# an example: loading the skip dataframe and listing the possible reasons
skip_df = pd.read_csv(Path(DATA_DIR) / Path('PANDA_Suspicious_Slides_15_05_2020.csv'))
print("possible faulty slide reasons", skip_df['reason'].unique())

fold_df = PandasDataLoader(images_csv_path=Path(DATA_DIR) / Path('train.csv'),
                           skip_csv=Path(DATA_DIR) / Path('PANDA_Suspicious_Slides_15_05_2020.csv'), 
                           skip_list=[])

# we create a possible stratification here, the options are by isup grade, or further distilled by isup grade and data provider
# stratified_isup_sample or stratified_isup_dp_sample, we use the former.

fold_df = fold_df.stratified_isup_sample(NFOLDS, SEED)

# we can create training/validation splits from the fold column
train_df = fold_df[fold_df['split'] != TRAIN_FOLD]
valid_df = fold_df[fold_df['split'] == TRAIN_FOLD]

display(train_df)
display(valid_df)

possible faulty slide reasons ['marks' 'No Mask' 'Background only'
 'No cancerous tissue but ISUP Grade > 0' 'tiss' 'blank']
********************
The training dataframe shape before filtering:(10616, 4)
The skip dataframe has shape: (675, 2), with reasons ['marks', 'No Mask', 'Background only', 'No cancerous tissue but ISUP Grade > 0', 'tiss', 'blank']
Filtering based on the following columns: ['marks', 'No Mask', 'Background only', 'No cancerous tissue but ISUP Grade > 0', 'tiss', 'blank']
number of duplicates in the skip df: (13, 2)
Training dataframe after filtering: (9954, 4)
Number of rows removed by filter: 662
********************


,image_id,data_provider,isup_grade,gleason_score,split
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,1
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,3
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,2
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,3
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,1
...,...,...,...,...,...
10622,ffcd99c47e57ad2934dc6bbf5edf6675,karolinska,0,0+0,2
10624,ffd2841373b39792ab0c84cccd066e31,radboud,0,negative,2
10625,ffdc59cd580a1468eac0e6a32dd1ff2d,radboud,5,4+5,2
10626,ffe06afd66a93258f8fabdef6044e181,radboud,0,negative,3


,image_id,data_provider,isup_grade,gleason_score,split
5,002a4db09dad406c85505a00fb6f6144,karolinska,0,0+0,0
6,003046e27c8ead3e3db155780dc5498e,karolinska,1,3+3,0
10,00412139e6b04d1e1cee8421f38f6e90,karolinska,0,0+0,0
13,004f6b3a66189b4e88b6a01ba19d7d31,karolinska,1,3+3,0
29,00c15b23b30a5ba061358d9641118904,radboud,5,4+5,0
...,...,...,...,...,...
10596,ff339e5fa7be6af83c1b43796092398f,karolinska,4,4+4,0
10603,ff596d5292ab979e9ba7291d0743b3fb,karolinska,0,0+0,0
10618,ffc005d56a21efbd034425623f596984,karolinska,2,3+4,0
10621,ffcbb41626c9267c5c20c4804bd5639a,radboud,4,3+5,0


In [5]:
from preprocessing.augmentations import StainAugment
import albumentations as A
from albumentations import (
    Flip, ShiftScaleRotate, RandomRotate90,
    Blur, RandomBrightnessContrast, 
    Compose, RandomScale, ElasticTransform, GaussNoise, GaussianBlur
)

def aug_routine(image):
    R = Compose([RandomRotate90(p=0.5), Flip(p=0.5)])
    #S = Compose([RandomScale(scale_limit=0.1, interpolation=1, p=0.5)])
    C = Compose([StainAugment(p=0.9)])
    #E = Compose([ElasticTransform(alpha=1, sigma=50, alpha_affine=50, interpolation=1, border_mode=4, p=0.5)])
    H = Compose([RandomBrightnessContrast(brightness_limit=0.03, contrast_limit=0.03, brightness_by_max=True, p=0.5)])
    B = Compose([GaussianBlur(blur_limit=5, p=0.5)])
    G = Compose([GaussNoise(var_limit=(10.0, 50.0), mean=0, p=0.5)])
    augment = Compose([C], p=0.9)
    aug_op = augment(image=image)
    image = aug_op['image']
    return image

## Tensorflow per-image data augmentations

In [ ]:
def flip(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_flip_left_right(x, seed=1)
    x = tf.image.random_flip_up_down(x, seed=1)

    return x

def rotate(x: tf.Tensor) -> tf.Tensor:
    # Rotate 0, 90, 180, 270 degrees
    return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32, seed=1))


augments = [flip]#, rotate]